# Разделение видео файла

К сожалению, пришлось отказаться от сжатия файлов, так как очень сильно нагружало систему и занимало очень долгое время. Было решено делить файл на части и отправлять частями через бота. Данный ноутбук является тестированием деления файла на части.

### Скачивание файла

In [38]:
import pytube as pt
yt = pt.YouTube(link)
t = yt.streams.filter(only_audio=True)
filename = t[0].download()

### Функция определения времени видео

In [39]:
import subprocess
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

### Определяем размер файла и вычсляем на сколько частей надо его делить

In [58]:
filename = 'test.mp4'

In [61]:
import os
# get file size in python
file_stats = os.stat(filename)
file_size = file_stats.st_size / (1024 * 1024)
print(f'File Size in MegaBytes is {file_size}')

File Size in MegaBytes is 104.3241491317749


In [62]:
import math

timesplit = math.ceil(file_size/45)
timesplit

3

### Вычисление времени видео и времени одного куска видео

In [67]:
videoLength = get_length(filename)
videoLength

17939.551202

In [69]:
splitLength = videoLength/timesplit
splitLength

5979.850400666666

In [78]:
starttime = [i * splitLength for i in range(timesplit)]
endtime = [(i+1) * splitLength for i in range(timesplit)]
starttime

[0.0, 5979.850400666666, 11959.700801333332]

In [79]:
endtime

[5979.850400666666, 11959.700801333332, 17939.551202]

### Делим видео на части

Была выбрана библиотека moviepy по причине того, что это самый быстрый способ поделить видео

In [83]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

targetname, _ = os.path.splitext(filename)
for i in range(timesplit):
    ffmpeg_extract_subclip(filename, starttime[i], endtime[i], targetname=f'{targetname}_part{i}.mp4')

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
